# pycaret을 이용한 가스공급량 예측
* 2019년 test 데이터 : '../data/test.csv'
* 2019년 제출 파일 : '../data/sample_submission.csv'
* 2013-2018년 가스공급량과 기온 자료 : '../2013-2018년_가스공급량_기온_습도_기압02.csv'
    * 이상치 제거(04_가스_기상청자료_합치기_이상치처리_ver0.2(2021.12.07).ipynb)
    * 365310개
* 2019년 예측 기온, 기압, 습도 자료 : '2019년_예측_기온_기압_습도.csv'
* 2019년 외부데이터 값 설명
    * 기온 : cat, lgbm, xgb 예측 평균
    * 기압, 습도 : dt, et, rf 예측 평균
* 특성
    * 일시 음력으로 변환
    * 'lunar_month', 'lunar_day', 'weekday', '시간'으로 기온 예측
    * 기온, 기압, 습도 순서로 예측
    * 각 특성 예측 마다 앞에서 예측한 특성 활용
    * 공급량 예측(6개) : 'lunar_month', 'weekday', '시간', '구분', '기온', '기압'
* pycaret 설정
    * combine rare level(high cardinality가진 특성 끼리 합쳐준다)
    * transformation(표준 정규 분포를 따르도록 수정)
    * robustscaler 적용(4분위 값 기준으로 데이터 배열)
    * 튜닝X, finalize_model O
* cat, lgbm 예측 평균
* 구분별 훈련/예측

In [1]:
# 제출 파일명 : sub56_ver4.7.csv
# 최고점 파일과 비교 점수(NMAE/R2)(sub48_ver3.9.csv) : 0.079985, 0.82211
# private 제출 점수 : X

## 데이터 가져오기

In [2]:
# 2019년 test 데이터(음력포함) : '2019년_음력포함_test.csv'
# 2019년 제출 파일 : '../data/sample_submission.csv'
# 2013-2018년 가스공급량과 기상청 자료(음력포함) : '2013-2018년_가스공급량_기온_습도_기압02.csv'

In [3]:
from pycaret.regression import *

In [4]:
import time
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm

### 2013-2018년 가스공급량과 기온, 기압, 습도, 음력 자료

In [5]:
total = pd.read_csv('2013-2018년_가스공급량_기온_습도_기압02.csv')
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365308 entries, 0 to 365307
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   일시           365308 non-null  object 
 1   year         365308 non-null  int64  
 2   month        365308 non-null  int64  
 3   day          365308 non-null  int64  
 4   weekday      365308 non-null  int64  
 5   시간           365308 non-null  int64  
 6   구분           365308 non-null  int64  
 7   공급량          365308 non-null  float64
 8   기온           365308 non-null  float64
 9   습도           365308 non-null  float64
 10  기압           365308 non-null  float64
 11  연월일          365308 non-null  object 
 12  hour         365308 non-null  int64  
 13  quarter      365308 non-null  int64  
 14  음력일시         365308 non-null  object 
 15  lunar_year   365308 non-null  int64  
 16  lunar_month  365308 non-null  int64  
 17  lunar_day    365308 non-null  int64  
dtypes: float64(4), int64(11)

In [6]:
total.head()

,일시,year,month,day,weekday,시간,구분,공급량,기온,습도,기압,연월일,hour,quarter,음력일시,lunar_year,lunar_month,lunar_day
0,2013-01-01 01,2013,1,1,1,1,0,2497.129,-8.5,57.0,1010.0,2013-01-01,1,1,2012-11-20,2012,11,20
1,2013-01-01 02,2013,1,1,1,2,0,2363.265,-8.4,60.0,1009.4,2013-01-01,2,1,2012-11-20,2012,11,20
2,2013-01-01 03,2013,1,1,1,3,0,2258.505,-8.1,58.0,1009.2,2013-01-01,3,1,2012-11-20,2012,11,20
3,2013-01-01 04,2013,1,1,1,4,0,2243.969,-8.2,58.0,1008.2,2013-01-01,4,1,2012-11-20,2012,11,20
4,2013-01-01 05,2013,1,1,1,5,0,2344.105,-8.2,61.0,1007.3,2013-01-01,5,1,2012-11-20,2012,11,20


### 2019년 테스트 데이터(음력포함)

In [7]:
test2019 = pd.read_csv('2019년_예측_기온_기압_습도02.csv')
test2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일자|시간|구분     15120 non-null  object 
 1   연월일          15120 non-null  object 
 2   시간           15120 non-null  int64  
 3   구분           15120 non-null  int64  
 4   year         15120 non-null  int64  
 5   quarter      15120 non-null  int64  
 6   month        15120 non-null  int64  
 7   day          15120 non-null  int64  
 8   weekday      15120 non-null  int64  
 9   음력일시         15120 non-null  object 
 10  lunar_year   15120 non-null  int64  
 11  lunar_month  15120 non-null  int64  
 12  lunar_day    15120 non-null  int64  
 13  기온           15120 non-null  float64
 14  기압           15120 non-null  float64
 15  습도           15120 non-null  float64
dtypes: float64(3), int64(10), object(3)
memory usage: 1.8+ MB


In [8]:
test2019.head()

,일자|시간|구분,연월일,시간,구분,year,quarter,month,day,weekday,음력일시,lunar_year,lunar_month,lunar_day,기온,기압,습도
0,2019-01-01 01 A,2019-01-01,1,0,2019,1,1,1,1,2018-11-26,2018,11,26,-3.652919,1016.246009,52.633333
1,2019-01-01 02 A,2019-01-01,2,0,2019,1,1,1,1,2018-11-26,2018,11,26,-3.984103,1016.867334,54.236667
2,2019-01-01 03 A,2019-01-01,3,0,2019,1,1,1,1,2018-11-26,2018,11,26,-4.492638,1017.684342,58.560000
3,2019-01-01 04 A,2019-01-01,4,0,2019,1,1,1,1,2018-11-26,2018,11,26,-4.799530,1017.739344,60.313333
4,2019-01-01 05 A,2019-01-01,5,0,2019,1,1,1,1,2018-11-26,2018,11,26,-5.053949,1008.569324,57.790000


## 2019년 공급량 훈련/예측, 구분별

In [9]:
# 구분에 따라 예측하기
divisions = test2019['구분'].unique()

# 학습 특성 : 'lunar_month', 'weekday', '시간', '구분', '기온', '기압'
# 타겟 특성 : '공급량'
test_col = ['lunar_month', 'weekday', '시간', '구분', '기온', '기압']
target_col = ['공급량']
train_col = test_col + target_col

gas2019_pred = pd.DataFrame()
for division in tqdm(divisions):
    # 구분별 훈련, 테스트 셋 설정
    temp_train = total[total['구분'] == division].reset_index(drop=True)
    temp_train = temp_train[train_col]

    temp_test = test2019[test2019['구분'] == division].reset_index(drop=True)
    temp_test = temp_test[test_col]

    # pycaret 설정
    exp = setup(temp_train, target=target_col[0], session_id=0, silent=True,
     numeric_features=['lunar_month', 'weekday', '시간'], combine_rare_levels=True,
     normalize=True, normalize_method='robust',
     transformation=True, transform_target=True,
     fold=7, fold_shuffle=True, verbose=False)

    # 모델 생성
    cat = create_model('catboost')
    lgbm = create_model('lightgbm')

    # 최종 훈련
    final_cat = finalize_model(cat)
    final_lgbm = finalize_model(lgbm)

    # 공급량 예측
    gas2019_pred_cat = predict_model(final_cat, data=temp_test)
    gas2019_pred_lgbm = predict_model(final_lgbm, data=temp_test)

    # 모델 공급량 예측 평균
    gas2019_division_pred = (gas2019_pred_cat['Label'] + gas2019_pred_lgbm['Label']) / 2

    # 구분별 예측 값 합치기
    gas2019_pred = pd.concat([gas2019_pred, gas2019_division_pred], ignore_index=True)
    # 00:11:38

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,17.7096,684.2938,26.1590,0.9771,0.1113,0.0810
1,17.5601,673.0692,25.9436,0.9768,0.1102,0.0801
2,17.2080,623.1105,24.9622,0.9796,0.1098,0.0798
3,16.9588,617.8013,24.8556,0.9792,0.1083,0.0795
4,17.3425,652.5583,25.5452,0.9781,0.1148,0.0818
5,17.1180,649.8125,25.4914,0.9776,0.1115,0.0807
6,17.3333,664.0042,25.7683,0.9769,0.1124,0.0810
Mean,17.3186,652.0928,25.5322,0.9779,0.1112,0.0806
SD,0.2372,22.8038,0.4476,0.0010,0.0019,0.0007


100%|██████████| 7/7 [10:53<00:00, 93.36s/it]


In [10]:
# 예측 결과 확인
gas2019_pred.head()

,0
0,1977.766221
1,1698.120964
2,1615.827928
3,1665.901044
4,1915.796879


## 제출 파일 가져오기

In [11]:
sub = pd.read_csv('../data/sample_submission.csv')
sub.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [12]:
sub['공급량'] = gas2019_pred
sub.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,1977.766221
1,2019-01-01 02 A,1698.120964
2,2019-01-01 03 A,1615.827928
3,2019-01-01 04 A,1665.901044
4,2019-01-01 05 A,1915.796879


In [13]:
sub.tail()

,일자|시간|구분,공급량
15115,2019-03-31 20 H,308.347669
15116,2019-03-31 21 H,315.625353
15117,2019-03-31 22 H,293.604112
15118,2019-03-31 23 H,266.547438
15119,2019-03-31 24 H,257.867374


In [14]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   일자|시간|구분  15120 non-null  object 
 1   공급량       15120 non-null  float64
dtypes: float64(1), object(1)
memory usage: 236.4+ KB


## 제출 파일 출력

In [15]:
sub.to_csv("sub56_ver4.7.csv", index=False)